<a href="https://colab.research.google.com/github/ReisK1ng/Projeto404/blob/main/ProjetoBanco.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

Aluno: Rafael Leonardo dos Reis Freitas

RGM: 37743449

Curso: Ciência da Computação

Turma: D1

Link do repositório Git


In [ ]:
# ========================
# Exceções (Capítulo 12)
# ========================
class ErroDeConta(Exception):
    """Classe base para exceções do sistema bancário"""
    pass  # Exceções específicas herdarão desta classe para tratamento genérico

class SaldoInsuficienteError(ErroDeConta):
    """Exceção para quando não há saldo suficiente"""
    pass

class OperacaoInvalidaError(ErroDeConta):
    """Exceção para operações inválidas"""
    pass

class ContaNaoEncontradaError(ErroDeConta):
    """Exceção para quando uma conta não é encontrada"""
    pass


# ========================
# Cliente (Capítulo 7)
# ========================
class Cliente:
    """Classe que representa um cliente do banco"""

    def __init__(self, nome, cpf):
        """Inicializa cliente com nome e CPF"""
        self.__nome = nome
        self.__cpf = cpf

    def __str__(self):
        """Representação em string do cliente para exibição legível"""
        return f'{self.__nome} ({self.__cpf})'

    @property
    def nome(self):
        """Retorna o nome do cliente"""
        return self.__nome

    @property
    def cpf(self):
        """Retorna o CPF do cliente"""
        return self.__cpf


# ========================
# Conta (Capítulos 7, 8, 10, 11)
# ========================
from abc import ABC, abstractmethod

class Conta(ABC):
    """Classe abstrata base para contas bancárias"""

    _total_contas = 0  # Contador de contas criadas

    def __init__(self, numero, cliente, saldo=0.0):
        """Inicializa conta com número, cliente e saldo opcional"""
        if not isinstance(cliente, Cliente):
            raise OperacaoInvalidaError("Cliente inválido")
        self._numero = numero
        self._cliente = cliente
        self.__saldo = max(saldo, 0.0)  # Garante saldo não negativo
        Conta._total_contas += 1

    @property
    def numero(self):
        """Retorna o número da conta"""
        return self._numero

    @property
    def cliente(self):
        """Retorna o cliente da conta"""
        return self._cliente

    @property
    def saldo(self):
        """Retorna o saldo da conta"""
        return self.__saldo

    @saldo.setter
    def saldo(self, valor):
        """Define o saldo da conta com validação"""
        if valor < 0:
            raise OperacaoInvalidaError("Saldo não pode ser negativo")
        self.__saldo = valor

    @classmethod
    def total_contas(cls):
        """Retorna o total de contas criadas"""
        return cls._total_contas

    def depositar(self, valor):
        """Realiza depósito na conta"""
        if valor <= 0:
            raise OperacaoInvalidaError("Valor de depósito deve ser positivo")
        self.__saldo += valor

    def transferir(self, destino, valor):
        """Transfere valor para outra conta"""
        if not isinstance(destino, Conta):
            raise OperacaoInvalidaError("Conta de destino inválida")
        self.sacar(valor)
        destino.depositar(valor)

    def extrato(self):
        """Retorna extrato da conta"""
        return f'Conta {self._numero} - {self._cliente.nome} - Saldo: R${self.__saldo:.2f}'

    @abstractmethod
    def sacar(self, valor):
        """Método abstrato para saque - implementado nas subclasses"""
        pass

    def atualiza(self, taxa):
        """Atualiza saldo com taxa de rendimento"""
        if taxa < 0:
            raise OperacaoInvalidaError("Taxa não pode ser negativa")
        self.__saldo += self.__saldo * taxa


class ContaCorrente(Conta):
    """Classe para conta corrente com taxa de depósito"""

    def sacar(self, valor):
        """Realiza saque na conta corrente"""
        if valor <= 0:
            raise OperacaoInvalidaError("Valor de saque deve ser positivo")
        if valor > self.saldo:
            raise SaldoInsuficienteError("Saldo insuficiente")
        self.saldo -= valor

    def depositar(self, valor):
        """Realiza depósito com taxa de R$0.10"""
        super().depositar(valor - 0.10)

    def atualiza(self, taxa):
        """Atualiza saldo com o dobro da taxa"""
        super().atualiza(taxa * 2)


class ContaPoupanca(Conta):
    """Classe para conta poupança com maior rendimento"""

    def sacar(self, valor):
        """Realiza saque na conta poupança"""
        if valor <= 0:
            raise OperacaoInvalidaError("Valor de saque deve ser positivo")
        if valor > self.saldo:
            raise SaldoInsuficienteError("Saldo insuficiente")
        self.saldo -= valor

    def atualiza(self, taxa):
        """Atualiza saldo com o triplo da taxa"""
        super().atualiza(taxa * 3)


# ========================
# Banco (Capítulo 13)
# ========================
from collections.abc import Iterable, Sized
import sqlite3
import os

class Banco(Iterable, Sized):
    """Classe que gerencia contas bancárias com persistência em SQLite"""

    def __init__(self, db_name="banco.db"):
        """Inicializa banco com conexão ao SQLite"""
        self.__db_name = db_name
        self.__setup_database()

    def __setup_database(self):
        """Configura tabelas no banco de dados com verificação de integridade"""
        with sqlite3.connect(self.__db_name) as conn:
            cursor = conn.cursor()

            # Verifica se a tabela antiga existe
            cursor.execute("SELECT name FROM sqlite_master WHERE type='table' AND name='contas'")
            tabela_antiga = cursor.fetchone()

            if tabela_antiga:
                # Verifica a estrutura da tabela antiga
                cursor.execute("PRAGMA table_info(contas)")
                colunas = [info[1] for info in cursor.fetchall()]

                if 'cpf_cliente' not in colunas:
                    # Se a tabela antiga existe sem a coluna cpf_cliente, recria tudo
                    cursor.execute("DROP TABLE IF EXISTS clientes")
                    cursor.execute("DROP TABLE IF EXISTS contas")
                    conn.commit()

            # Cria as tabelas com a nova estrutura
            cursor.execute("""
                CREATE TABLE IF NOT EXISTS clientes (
                    cpf TEXT PRIMARY KEY,
                    nome TEXT NOT NULL
                )
            """)
            cursor.execute("""
                CREATE TABLE IF NOT EXISTS contas (
                    numero INTEGER PRIMARY KEY,
                    cpf_cliente TEXT NOT NULL,
                    tipo TEXT NOT NULL,
                    saldo REAL NOT NULL,
                    FOREIGN KEY (cpf_cliente) REFERENCES clientes(cpf)
                )
            """)
            conn.commit()

    def __conectar(self):
        """Retorna conexão com o banco de dados SQLite"""
        return sqlite3.connect(self.__db_name)

    def adicionar_conta(self, conta):
        """Adiciona uma nova conta ao banco e seu cliente, se necessário"""
        with self.__conectar() as conn:
            cursor = conn.cursor()

            # Verifica se cliente já existe
            cursor.execute("SELECT 1 FROM clientes WHERE cpf = ?", (conta.cliente.cpf,))
            if not cursor.fetchone():
                cursor.execute(
                    "INSERT INTO clientes (cpf, nome) VALUES (?, ?)",
                    (conta.cliente.cpf, conta.cliente.nome)
                )

            # Adiciona conta
            cursor.execute(
                "INSERT INTO contas (numero, cpf_cliente, tipo, saldo) VALUES (?, ?, ?, ?)",
                (conta.numero, conta.cliente.cpf, conta.__class__.__name__, conta.saldo)
            )
            conn.commit()

    def buscar_conta(self, numero):
        """Busca conta pelo número e reconstrói objeto Python a partir dos dados"""
        with self.__conectar() as conn:
            cursor = conn.cursor()
            cursor.execute("""
                SELECT c.numero, cl.nome, cl.cpf, c.tipo, c.saldo
                FROM contas c
                JOIN clientes cl ON c.cpf_cliente = cl.cpf
                WHERE c.numero = ?
            """, (numero,))
            dados = cursor.fetchone()

            if not dados:
                raise ContaNaoEncontradaError(f"Conta {numero} não encontrada")

            cliente = Cliente(dados[1], dados[2])
            if dados[3] == "ContaCorrente":
                conta = ContaCorrente(dados[0], cliente, dados[4])
            elif dados[3] == "ContaPoupanca":
                conta = ContaPoupanca(dados[0], cliente, dados[4])
            else:
                raise OperacaoInvalidaError(f"Tipo de conta inválido: {dados[3]}")

            return conta

    def listar_contas(self):
        """Lista todas as contas cadastradas no banco"""
        with self.__conectar() as conn:
            cursor = conn.cursor()
            cursor.execute("""
                SELECT c.numero, cl.nome, cl.cpf, c.tipo, c.saldo
                FROM contas c
                JOIN clientes cl ON c.cpf_cliente = cl.cpf
                ORDER BY c.numero
            """)
            return cursor.fetchall()

    def atualizar_saldo(self, numero, saldo):
        """Atualiza o saldo de uma conta no banco de dados"""
        with self.__conectar() as conn:
            cursor = conn.cursor()
            cursor.execute(
                "UPDATE contas SET saldo = ? WHERE numero = ?",
                (saldo, numero)
            )
            conn.commit()

    def remover_conta(self, numero):
        """Remove uma conta do banco de dados"""
        with self.__conectar() as conn:
            cursor = conn.cursor()
            cursor.execute("DELETE FROM contas WHERE numero = ?", (numero,))
            conn.commit()

    def __iter__(self):
        """Permite iteração sobre as contas do banco (for conta in banco)"""
        contas = self.listar_contas()
        for conta_data in contas:
            yield self.buscar_conta(conta_data[0])

    def __len__(self):
        """Permite usar len(banco) para saber número de contas cadastradas"""
        with self.__conectar() as conn:
            cursor = conn.cursor()
            cursor.execute("SELECT COUNT(*) FROM contas")
            return cursor.fetchone()[0]


# ========================
# Interface (menu CLI)
# ========================
def menu():
    """Interface de linha de comando para o sistema bancário"""
    banco = Banco()  # Instancia o gerenciador do banco

    while True:
        # Exibe menu principal
        print("\n==== MENU BANCÁRIO ====")
        print("1. Criar conta")
        print("2. Depositar")
        print("3. Sacar")
        print("4. Transferir")
        print("5. Listar contas")
        print("6. Remover conta")
        print("7. Sair")

        try:
            opcao = input("Escolha uma opção: ").strip()

            if opcao == "1":
                # Criar nova conta
                nome = input("Nome do cliente: ").strip()
                cpf = input("CPF (apenas números): ").strip()
                tipo = input("Tipo (corrente/poupança): ").strip().lower()
                numero = int(input("Número da conta: "))
                saldo_inicial = float(input("Saldo inicial: R$ "))

                cliente = Cliente(nome, cpf)

                if tipo == "corrente":
                    conta = ContaCorrente(numero, cliente, saldo_inicial)
                elif tipo == "poupança":
                    conta = ContaPoupanca(numero, cliente, saldo_inicial)
                else:
                    print("Tipo de conta inválido!")
                    continue

                banco.adicionar_conta(conta)
                print(f"Conta {numero} criada com sucesso!")

            elif opcao == "2":
                # Depositar
                numero = int(input("Número da conta: "))
                valor = float(input("Valor a depositar: R$ "))

                conta = banco.buscar_conta(numero)
                conta.depositar(valor)
                banco.atualizar_saldo(numero, conta.saldo)
                print(f"Depósito de R$ {valor:.2f} realizado com sucesso!")

            elif opcao == "3":
                # Sacar
                numero = int(input("Número da conta: "))
                valor = float(input("Valor a sacar: R$ "))

                conta = banco.buscar_conta(numero)
                conta.sacar(valor)
                banco.atualizar_saldo(numero, conta.saldo)
                print(f"Saque de R$ {valor:.2f} realizado com sucesso!")

            elif opcao == "4":
                # Transferir
                origem = int(input("Conta de origem: "))
                destino = int(input("Conta de destino: "))
                valor = float(input("Valor a transferir: R$ "))

                conta_origem = banco.buscar_conta(origem)
                conta_destino = banco.buscar_conta(destino)

                conta_origem.transferir(conta_destino, valor)

                banco.atualizar_saldo(origem, conta_origem.saldo)
                banco.atualizar_saldo(destino, conta_destino.saldo)

                print(f"Transferência de R$ {valor:.2f} realizada com sucesso!")

            elif opcao == "5":
                # Listar contas
                print("\n=== LISTA DE CONTAS ===")
                for conta_data in banco.listar_contas():
                    print(f"Conta {conta_data[0]} - {conta_data[1]} ({conta_data[2]})")
                    print(f"  Tipo: {conta_data[3]} - Saldo: R$ {conta_data[4]:.2f}")
                print("======================")

            elif opcao == "6":
                # Remover conta
                numero = int(input("Número da conta a remover: "))
                banco.remover_conta(numero)
                print(f"Conta {numero} removida com sucesso!")

            elif opcao == "7":
                # Sair do sistema
                print("Encerrando sistema bancário...")
                break

            else:
                print("Opção inválida! Tente novamente.")

        except ValueError:
            print("Erro: Valor inválido fornecido!")
        except SaldoInsuficienteError as e:
            print(f"Erro: {e}")
        except OperacaoInvalidaError as e:
            print(f"Erro: {e}")
        except ContaNaoEncontradaError as e:
            print(f"Erro: {e}")
        except Exception as e:
            print(f"Erro inesperado: {e}")


if __name__ == "__main__":
    menu()